## 사전 준비: 라이브러리 및 데이터 로드


In [1]:
# 런타임 다시 시작 후, 이 셀을 실행하여 라이브러리와 폰트를 로드합니다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm

# 한글 폰트 설정
plt.rc('font', family='NanumGothic')
# plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

In [2]:
# 실습용 'tips' 데이터셋을 불러옵니다.
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### 📝 Statsmodels 사용법

1.  **데이터 분리 (X, y)**: 분석할 데이터를 **독립변수 `X`**(설명하는 변수들, DataFrame)와 **종속변수 `y`**(알고싶은 결과, Series)로 직접 나눕니다.
2.  **데이터 가공**:
    - **더미 변수 생성**: `pd.get_dummies()` 함수를 사용해 문자(범주형) 데이터를 0과 1로 이루어진 숫자 데이터로 직접 변환합니다.
    - **상수항(Intercept) 추가**: **가장 중요한 단계입니다.** 회귀식의 y절편(`b`ใน `y=ax+b`)을 계산하기 위해 `sm.add_constant(X)`를 사용하여 X 데이터에 상수항 컬럼을 **수동으로 추가**해야 합니다.
3.  **모델 학습**: `sm.OLS(y, X)` 또는 `sm.Logit(y, X)` 함수에 준비된 `y`와 `X` 데이터를 전달하여 모델을 학습시킵니다.

이제 이 3단계를 따라 모든 문제를 해결해 봅시다.


---


## 문제 1 (난이도: 하): 다중회귀분석의 첫걸음

> **🎯 목표:** 여러 개의 원인(독립변수)을 동시에 고려하여 결과를 예측하는 **다중회귀분석**을 파이썬 배열 방식으로 구현합니다.


### 💡 핵심 개념:

단순회귀가 하나의 원인(`X`)으로 결과를 설명했다면, **다중회귀**는 **여러 개의 원인(`X1, X2, ...`)**을 동시에 고려하여 결과를 더 정교하게 설명하는 방법입니다. 이때 각 원인의 영향력(계수)은 **'다른 원인들이 모두 동일하다고 통제했을 때'**의 순수한 영향력을 의미합니다.


### 📌 수행 과제:

1. `tips` 데이터셋에서 `tip`을 종속변수(y)로, `total_bill`과 `size`를 독립변수(X)로 분리하세요.
2. 독립변수(X)에 상수항을 추가하세요.
3. `sm.OLS`를 사용하여 다중회귀 모델을 학습시키고, `.summary()`로 결과를 확인하세요.
4. 결과표를 보고 `total_bill`과 `size`의 계수(coef)가 각각 무엇을 의미하는지 해석해 보세요.


### ✍️ 코드 작성:


In [ ]:
# 1. 여기에 데이터를 y와 X로 분리하는 코드를 작성하세요.
# 종속변수 y에 'tip' 컬럼을 할당하세요.
y = tips[...]

# 독립변수 X에 'total_bill'과 'size' 컬럼을 리스트로 묶어 할당하세요.
X = tips[[..., ...]]

# 2. 여기에 상수항을 추가하는 코드를 작성하세요.
# sm.add_constant()를 사용해 X 데이터에 상수항을 추가합니다.
X_const = sm.add_constant(...)

# 3. 여기에 모델을 학습하고 결과를 출력하는 코드를 작성하세요.
# sm.OLS() 함수에 y와 X_const를 순서대로 넣어 모델을 만들고 학습시킵니다.
model_1 = sm.OLS(..., ...).fit()
print(model_1.summary())

# 4. 결과 해석 (아래 주석에 직접 작성해 보세요)
# total_bill 계수의 의미:
# size 계수의 의미:

### 🤔 생각해 볼 문제:

`X_const.head()`를 출력해서 `sm.add_constant()`가 실제로 X 데이터에 어떤 변화를 주었는지 직접 눈으로 확인해보세요.


---


## 문제 2 (난이도: 하): 더미 변수 활용하기

> **🎯 목표:** '요일'과 같은 문자(범주형) 데이터를 분석에 활용하기 위해 **더미 변수**로 변환하는 방법을 배우고, 회귀분석에 적용합니다.


### 💡 핵심 개념:

**더미 변수**는 '성별', '요일'과 같은 문자(범주형) 데이터를 통계 모델에 사용하기 위해 **0과 1로 이루어진 숫자 변수**로 변환하는 기법입니다. Pandas의 `pd.get_dummies()` 함수를 사용하면 이 과정을 쉽게 처리할 수 있습니다. 이때, 여러 범주 중 하나는 **기준(reference)**이 되어 분석 오류를 막고 해석의 기준점 역할을 합니다.


### 📌 수행 과제:

1. `total_bill`과 '요일(`day`)'이 팁(`tip`)에 미치는 영향을 분석해봅시다.
2. `pd.get_dummies()`를 사용하여 'day' 컬럼을 더미 변수로 변환하세요. (단, `drop_first=True` 옵션 사용)
3. 기존 독립변수(`total_bill`)와 생성된 더미 변수를 합쳐 최종 X 데이터를 만드세요.
4. 상수항을 추가하고 회귀 모델을 학습시킨 후, 결과를 해석하세요.


### ✍️ 코드 작성:


In [ ]:
# 1. 기본 데이터를 분리합니다.
y = tips['tip']
X_base = tips[['total_bill']]

# 2. 여기에 'day' 컬럼으로 더미 변수를 생성하는 코드를 작성하세요.
# pd.get_dummies()를 사용하고, drop_first=True 옵션을 잊지 마세요.
day_dummies = pd.get_dummies(tips['day'], prefix='day', drop_first=...)

# 3. 여기에 기본 X 데이터와 더미 변수를 합치는 코드를 작성하세요.
# pd.concat을 사용하고, axis=1 옵션으로 옆으로 붙입니다.
X = pd.concat([X_base, ...], axis=1)

# 4. 상수항을 추가하고 모델을 학습시킨 후 결과를 출력하세요.
X_const = sm.add_constant(X)
model_2 = sm.OLS(y, X_const).fit()
print(model_2.summary())

# 5. 결과 해석 (아래 주석에 직접 작성해 보세요)
# 기준이 된 요일은 무엇인가?:
# 'day_Sat' 계수의 의미:

### 🤔 생각해 볼 문제:

`drop_first=True` 옵션을 빼고 `False`로 설정한 뒤 모델을 다시 학습시켜보세요. 결과가 어떻게 달라지나요? (힌트: `statsmodels`가 다중공선성에 대한 경고 메시지를 보여줄 수 있습니다.)


---


## 문제 3 (난이도: 중): 상호작용 효과 분석하기

> **🎯 목표:** 한 변수의 효과가 다른 변수의 상황에 따라 달라지는 **상호작용 효과**를 직접 변수를 곱하여 만들고, 그 의미를 해석합니다.


### 💡 핵심 개념:

**상호작용**은 한 변수의 효과가 다른 변수의 수준에 따라 달라지는 현상을 말합니다. 예를 들어, "식사 금액이 팁에 미치는 영향은 흡연 여부에 따라 다르다"와 같은 가설을 검증할 때 사용됩니다. 배열 방식에서는 상호작용 항을 **두 변수의 곱**으로 직접 만들어서 모델에 추가합니다.


### 📌 수행 과제:

1. "식사 총액(`total_bill`)이 팁에 미치는 영향은 흡연자(`smoker`) 그룹과 비흡연자 그룹에서 다를 것이다"라는 가설을 검증해봅시다.
2. `smoker` 변수를 더미 변수(`smoker_Yes`)로 만드세요. ('Yes' = 1, 'No' = 0)
3. `total_bill`과 `smoker_Yes`를 곱하여 상호작용 항(`bill_x_smoker`)을 만드세요.
4. `total_bill`, `smoker_Yes`, `bill_x_smoker`를 모두 포함하여 회귀 모델을 학습시키고 결과를 해석하세요.


### ✍️ 코드 작성:


In [ ]:
# 1. 기본 데이터를 준비합니다.
y = tips['tip']
X = tips[['total_bill']]

# 2. 여기에 'smoker' 더미 변수를 X에 추가하는 코드를 작성하세요.
X['smoker_Yes'] = pd.get_dummies(tips['smoker'], drop_first=True, dtype=int)

# 3. 여기에 상호작용 항을 직접 계산하여 X에 추가하는 코드를 작성하세요.
# 'total_bill' 컬럼과 'smoker_Yes' 컬럼을 곱합니다.
X['bill_x_smoker'] = X[...] * X[...]

# 4. 상수항을 추가하고 모델을 학습시킨 후 결과를 출력하세요.
X_const = sm.add_constant(X)
model_3 = sm.OLS(y, X_const).fit()
print(model_3.summary())

# 5. 결과 해석 (아래 주석에 직접 작성해 보세요)
# 상호작용 항(bill_x_smoker)의 p-value는 유의미한가?:
# 상호작용 항 계수의 의미:

### 🤔 생각해 볼 문제:

위 모델에서 비흡연자(smoker_Yes=0)의 `total_bill` 기울기는 얼마일까요? 그리고 흡연자(smoker_Yes=1)의 `total_bill` 기울기는 얼마일까요? (힌트: 흡연자의 기울기 = 기본 `total_bill` 기울기 + 상호작용 항 기울기)


---


## 문제 4 (난이도: 중): 로지스틱 회귀분석

> **🎯 목표:** 결과가 '예/아니오' 같은 범주형일 때 사용하는 **로지스틱 회귀분석**을 구현하고, 그 결과를 **오즈비(Odds Ratio)**로 해석하는 방법을 배웁니다.


### 💡 핵심 개념:

**로지스틱 회귀분석**은 결과가 '예/아니오', '성공/실패'처럼 **두 가지 중 하나**일 때 사용하는 분석 방법입니다. 어떤 조건에서 특정 결과가 나타날 **'확률'**을 예측합니다. 회귀 계수는 그대로 해석하기 어렵고, `np.exp(계수)`를 계산한 **오즈비(Odds Ratio)**로 해석해야 합니다. 오즈비가 1보다 크면 확률 증가, 1보다 작으면 확률 감소를 의미합니다.


### 📌 수행 과제:

1. 식사 총액(`total_bill`)과 팁(`tip`) 금액을 보고, 그 고객이 '남성'일 확률을 예측하는 로지스틱 회귀 모델을 만들어봅시다.
2. 종속변수 `y`를 'sex'가 'Male'이면 1, 아니면 0이 되도록 변환하세요.
3. `sm.Logit` 함수를 사용하여 모델을 학습시키고, 결과를 확인하세요.
4. `np.exp()`를 이용해 계수를 오즈비로 변환하고, 그 의미를 해석하세요.


### ✍️ 코드 작성:


In [ ]:
# 1. 여기에 종속변수 y를 0과 1로 변환하는 코드를 작성하세요.
# 'sex'가 'Male'이면 1, 아니면 0
y = tips['sex'].apply(lambda x: 1 if x == '...' else 0)

# 2. 독립변수 X를 준비하고 상수항을 추가하세요.
X = tips[['total_bill', 'tip']]
X_const = sm.add_constant(X)

# 3. 여기에 로지스틱 회귀 모델을 학습시키는 코드를 작성하세요.
# sm.Logit() 함수를 사용합니다.
model_4 = sm.Logit(..., ...).fit()
print(model_4.summary())

# 4. 여기에 오즈비를 계산하고 출력하는 코드를 작성하세요.
odds_ratios = np.exp(model_4.params)
print("\n--- 오즈비 (Odds Ratios) ---")
print(odds_ratios)

# 5. 결과 해석 (아래 주석에 직접 작성해 보세요)
# total_bill 오즈비의 의미:
# tip 오즈비의 의미:

### 🤔 생각해 볼 문제:

`model_4.predict(X_const)` 코드를 실행하면 각 손님에 대해 '남성일 확률'을 예측해줍니다. 이 예측 확률이 가장 높은 상위 5명의 손님 데이터(total_bill, tip)는 어떤 특징을 가지고 있는지 확인해보세요.


---


## 문제 5 (난이도: 중): 최고의 모델 찾기: AIC

> **🎯 목표:** 여러 모델 중 어떤 모델이 가장 좋은지 객관적인 점수로 평가하는 **AIC**의 개념을 이해하고, 이를 이용해 최적의 모델을 선택합니다.


### 💡 핵심 개념:

**AIC (Akaike Information Criterion)**는 여러 모델 중 어떤 모델이 가장 좋은지 평가하는 '점수'입니다. AIC는 **모델의 설명력(얼마나 잘 맞추는가)**은 높이고, **모델의 복잡성(얼마나 많은 변수를 썼는가)**에는 벌점을 줍니다. 따라서 **AIC 점수는 낮을수록 더 좋은(효율적인) 모델**이라고 평가합니다.


### 📌 수행 과제:

1. 팁(`tip`) 금액을 예측하기 위한 세 가지 다른 모델을 만듭니다.
   - 모델 A: `total_bill`만 사용
   - 모델 B: `total_bill`, `size` 사용
   - 모델 C: `total_bill`, `size`, `day`(더미) 사용
2. 각 모델의 `.aic` 속성을 확인하여 AIC 점수를 비교하세요.
3. 어떤 모델이 가장 좋은 모델인지 선택하고, 그 이유를 설명하세요.


### ✍️ 코드 작성:


In [ ]:
y = tips['tip']

# --- 모델 A: total_bill만 사용 ---
X_A = tips[['total_bill']]
X_A_const = sm.add_constant(X_A)
model_A = sm.OLS(y, X_A_const).fit()
print("Model A AIC:", model_A.aic)

# --- 모델 B: total_bill과 size 사용 ---
# 1. 여기에 모델 B의 X 데이터를 준비하고, 학습 후 AIC를 출력하는 코드를 작성하세요.
X_B = tips[[..., ...]]
X_B_const = sm.add_constant(X_B)
model_B = sm.OLS(y, X_B_const).fit()
print("Model B AIC:", ...)

# --- 모델 C: total_bill, size, day 사용 ---
# 2. 여기에 모델 C의 X 데이터를 준비하고, 학습 후 AIC를 출력하는 코드를 작성하세요.
# (힌트: 문제 2에서 X 데이터를 만들었던 과정을 참고하세요.)
day_dummies = pd.get_dummies(tips['day'], prefix='day', drop_first=True)
X_C = pd.concat([X_B, ...], axis=1)
X_C_const = sm.add_constant(X_C)
model_C = sm.OLS(y, X_C_const).fit()
print("Model C AIC:", ...)

# 3. 결과 해석 (아래 주석에 직접 작성해 보세요)
# 가장 좋은 모델(AIC가 가장 낮은 모델)은?:
# 그렇게 생각하는 이유:

### 🤔 생각해 볼 문제:

AIC는 모델의 복잡도(사용한 변수의 개수)에 벌점을 줍니다. 모델 B는 모델 A보다 변수가 하나 더 많음에도 AIC가 낮아졌습니다. 이는 무엇을 의미할까요?
